# Train MLP Baseline (toy example and hyperparameter tuning)

This notebook trains the `MLPBaseline` model from `model_classes/mlp_baseline.py` with a simple forward/backward loop and a small hyperparameter grid search.

Notes:
- For demonstration this uses a synthetic dataset (random features + labels). Replace the data-loading cell with your real dataset / `DataLoader` from `data_loaders.py` when ready.
- Training loop is minimal: forward, compute loss, backward, optimizer step.
- Hyperparameter tuning is a simple grid search over a few combos; it's not parallelized.

In [8]:
import os
import sys

# Assuming you start the notebook from the training_scripts/ directory
PROJECT_ROOT = os.path.abspath("..")
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

print("Project root in sys.path:", PROJECT_ROOT)


Project root in sys.path: /home/quiet98k/Code/Fall25/hand-gestures-classifier


In [9]:
# Basic imports and helper utilities
import os
from itertools import product
from typing import Tuple, Dict, Any

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

# Import model factory and dataset from the repo
from model_classes.mlp_baseline import create_mlp_baseline
from data_loaders import LandmarksDataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)

Device: cuda


In [10]:
# Use the real LandmarksDataset from the repository
# We flatten landmarks for the MLP (shape: N_landmarks * 2)
torch.manual_seed(42)

# Create train/val dataset instances (assumes data/annotations/{train,val} exist)
annotations_root = "../data/annotations"
train_ds = LandmarksDataset(annotations_root=annotations_root, split='train', flatten=True)
val_ds = LandmarksDataset(annotations_root=annotations_root, split='val', flatten=True)

# Infer input_dim and num_classes from dataset (keeps things consistent)
first = train_ds[0]['landmarks']
if isinstance(first, torch.Tensor):
    input_dim = int(first.numel())
else:
    # Fallback: assume 42 features if something unusual happens
    input_dim = 42
num_classes = train_ds.num_classes

print('Dataset sizes: train=', len(train_ds), 'val=', len(val_ds))
print('Input dim (flattened landmarks):', input_dim, 'Num classes:', num_classes)

# Create DataLoaders
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True, drop_last=False)
val_loader = DataLoader(val_ds, batch_size=64, shuffle=False, drop_last=False)

# Quick smoke test: retrieve one batch and print basic shapes
try:
    batch = next(iter(train_loader))
    print('Successfully retrieved a batch from train_loader')
except Exception as e:
    print('Error when iterating train_loader:', repr(e))
    raise

# Extract landmarks and labels from batch
if isinstance(batch, dict):
    xb = batch['landmarks']
    yb = batch['label']
else:
    xb, yb = batch

print('Batch shapes:', getattr(xb, 'shape', None), getattr(yb, 'shape', None))

Dataset sizes: train= 408237 val= 53392
Input dim (flattened landmarks): 42 Num classes: 18
Successfully retrieved a batch from train_loader
Batch shapes: torch.Size([64, 42]) torch.Size([64])
Successfully retrieved a batch from train_loader
Batch shapes: torch.Size([64, 42]) torch.Size([64])


In [14]:
# Training and evaluation helpers
def train_epoch(model: nn.Module, loader: DataLoader, optimizer, criterion) -> Dict[str, float]:
    model.train()
    total_loss = 0.0
    total_correct = 0
    total = 0
    for batch in loader:
        # Support both dict-style batches (from our Dataset) and tuple batches
        if isinstance(batch, dict):
            xb = batch['landmarks']
            yb = batch['label']
        else:
            xb, yb = batch

        xb = xb.to(device)
        yb = yb.to(device)
        optimizer.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * xb.size(0)
        preds = logits.argmax(dim=1)
        total_correct += (preds == yb).sum().item()
        total += xb.size(0)

    return {
        'loss': total_loss / total,
        'acc': total_correct / total,
    }

def evaluate(model: nn.Module, loader: DataLoader, criterion) -> Dict[str, float]:
    model.eval()
    total_loss = 0.0
    total_correct = 0
    total = 0
    with torch.no_grad():
        for batch in loader:
            if isinstance(batch, dict):
                xb = batch['landmarks']
                yb = batch['label']
            else:
                xb, yb = batch

            xb = xb.to(device)
            yb = yb.to(device)
            logits = model(xb)
            loss = criterion(logits, yb)
            total_loss += loss.item() * xb.size(0)
            preds = logits.argmax(dim=1)
            total_correct += (preds == yb).sum().item()
            total += xb.size(0)

    return {
        'loss': total_loss / total,
        'acc': total_correct / total,
    }

In [15]:
# Simple hyperparameter grid search (small grid for demo)
results = []
save_dir = os.path.join(os.getcwd(), 'training_outputs')
os.makedirs(save_dir, exist_ok=True)

param_grid = {
    'lr': [1e-3, 5e-4],
    'hidden_dims': [(128, 64), (256, 128)],
    'dropout': [0.1, 0.2],
    'batchnorm': [False, True],
    # tune batch size and number of epochs as well
    'batch_size': [32, 64],
    'max_epochs': [8, 12],
}

# Loss (kept separate for clarity)
criterion = nn.CrossEntropyLoss()

# iterate grid
for lr, hidden_dims, dropout, batchnorm, batch_size, max_epochs in product(
    param_grid['lr'], param_grid['hidden_dims'], param_grid['dropout'], param_grid['batchnorm'],
    param_grid['batch_size'], param_grid['max_epochs']
):
    print('Running config:', {'lr': lr, 'hidden_dims': hidden_dims, 'dropout': dropout, 'batchnorm': batchnorm, 'batch_size': batch_size, 'max_epochs': max_epochs})

    # build model and loaders
    model = create_mlp_baseline(input_dim=input_dim, num_classes=num_classes, hidden_dims=hidden_dims, dropout=dropout, activation='relu', batchnorm=batchnorm)
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    # create DataLoaders directly (no helper)
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, drop_last=False)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, drop_last=False)

    best_val_acc = 0.0
    best_state: dict[str, Any] | None = None

    for epoch in range(1, max_epochs + 1):
        tr = train_epoch(model, train_loader, optimizer, criterion)
        va = evaluate(model, val_loader, criterion)
        print(f"Epoch {epoch}/{max_epochs}  train_loss={tr['loss']:.4f} train_acc={tr['acc']:.4f}  val_loss={va['loss']:.4f} val_acc={va['acc']:.4f}")

        # save best model for this config
        if va['acc'] > best_val_acc:
            best_val_acc = va['acc']
            best_state = {
                'model_state': model.state_dict(),
                'optimizer_state': optimizer.state_dict(),
                'epoch': epoch,
            }

    # persist best for this config
    config_name = f"mlp_lr{lr}_hd{hidden_dims[0]}-{hidden_dims[1]}_do{int(dropout*100)}_bn{int(batchnorm)}_bs{batch_size}_ep{max_epochs}"
    out_path = os.path.join(save_dir, config_name + '.pth')
    if best_state is not None:
        torch.save({'config': {'lr': lr, 'hidden_dims': hidden_dims, 'dropout': dropout, 'batchnorm': batchnorm, 'batch_size': batch_size, 'max_epochs': max_epochs}, 'best_val_acc': best_val_acc, 'state': best_state}, out_path)

    results.append({'config': {'lr': lr, 'hidden_dims': hidden_dims, 'dropout': dropout, 'batchnorm': batchnorm, 'batch_size': batch_size, 'max_epochs': max_epochs}, 'best_val_acc': best_val_acc, 'path': out_path})

# summarize results
results_sorted = sorted(results, key=lambda r: r['best_val_acc'], reverse=True)
print('Top results:')
for r in results_sorted[:5]:
    print(r['best_val_acc'], r['config'], r['path'])

Running config: {'lr': 0.001, 'hidden_dims': (128, 64), 'dropout': 0.1, 'batchnorm': False, 'batch_size': 32, 'max_epochs': 8}
Epoch 1/8  train_loss=0.3068 train_acc=0.8973  val_loss=0.0724 val_acc=0.9818
Epoch 1/8  train_loss=0.3068 train_acc=0.8973  val_loss=0.0724 val_acc=0.9818
Epoch 2/8  train_loss=0.0881 train_acc=0.9761  val_loss=0.0523 val_acc=0.9867
Epoch 2/8  train_loss=0.0881 train_acc=0.9761  val_loss=0.0523 val_acc=0.9867
Epoch 3/8  train_loss=0.0740 train_acc=0.9798  val_loss=0.0530 val_acc=0.9857
Epoch 3/8  train_loss=0.0740 train_acc=0.9798  val_loss=0.0530 val_acc=0.9857
Epoch 4/8  train_loss=0.0670 train_acc=0.9815  val_loss=0.0455 val_acc=0.9879
Epoch 4/8  train_loss=0.0670 train_acc=0.9815  val_loss=0.0455 val_acc=0.9879


KeyboardInterrupt: 

In [ ]:
# Load best model and do a quick inference check
best = results_sorted[0]
print('Best config:', best['config'], 'val_acc=', best['best_val_acc'])
ckpt = torch.load(best['path'], map_location=device)
cfg = ckpt['config']
model = create_mlp_baseline(input_dim=input_dim, num_classes=num_classes, hidden_dims=tuple(cfg['hidden_dims']), dropout=cfg['dropout'], activation='relu', batchnorm=cfg['batchnorm'])
model.load_state_dict(ckpt['state']['model_state'])
model = model.to(device).eval()

# run inference on a small batch
xb, yb = next(iter(val_loader))
with torch.no_grad():
    logits = model(xb.to(device))
    preds = logits.argmax(dim=1).cpu()

print('Sample preds:', preds[:10].tolist())
print('Sample labels:', yb[:10].tolist())